In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import base64
import mimetypes
from PIL import Image
from io import BytesIO
import pandas as pd
from io import StringIO
import fitz
load_dotenv()

True

In [2]:
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'))

In [30]:
os.mkdir('images/biedronka20112023-25112023')
doc = fitz.open("images/biedronka20112023-25112023.pdf")

for idx in range(doc.page_count):
    # Select the page you want to convert (0 for the first page)
    page = doc.load_page(idx)

    # Convert it to a pixmap (an image)
    pix = page.get_pixmap()

    # Save the image
    image_path = f"images/biedronka20112023-25112023/page{idx}.png"
    pix.save(image_path)

# Close the document
doc.close()

In [3]:
def chat(query, image_path):
    # Guess the MIME type of the image
    mime_type, _ = mimetypes.guess_type(image_path)
    if not mime_type or not mime_type.startswith('image'):
        raise ValueError("The file type is not recognized as an image")
    
    # Read the image binary data
    with open(image_path, 'rb') as image_file:
        image_data = image_file.read()
        encoded_string = base64.b64encode(image_data).decode('utf-8')
    
    # Format the result with the appropriate prefix
    image_base64 = f"data:{mime_type};base64,{encoded_string}"

    # Read image for size using PIL
    image = Image.open(BytesIO(image_data))
    width, height = image.size
    print("Width:", width, "Height:", height)
    
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": query},
                    {
                        "type": "image_url",
                        "image_url": {"url": image_base64, "detail": "high"}
                    },
                ],
            }
        ],
        max_tokens=300,
    )

    return response.choices[0].message.content

In [14]:
query = ''' You are given an image of a invoice in Polish.
            I need a list of items that is in the middle of a image. There are column names there. Return the column names as a list separated by / and return the values seprated by / and a new line if there are mulitple rows.
            Return only this format and nothing else.'''

In [15]:
response = chat(query , f'images/387459570_1070736067600198_8222554501514766161_n.jpg')
response

Width: 3024 Height: 4032


'Poz. / Nr art. / Oznaczenie / L. / Cena netto / Razem netto\n100002-PL / Windows 7 Professional / 1 / 68,28 zł / 68,28 zł'

In [16]:
print(response)

Poz. / Nr art. / Oznaczenie / L. / Cena netto / Razem netto
100002-PL / Windows 7 Professional / 1 / 68,28 zł / 68,28 zł


In [9]:
offer = ''
for row in response.split('\n'):
    if '/' in row and len(row.split('/')) == 2:
        offer += row + '\n'


In [11]:
# Using StringIO to simulate a file-like object
string_io = StringIO(offer.strip())

# Reading the string into a pandas DataFrame
df = pd.read_csv(string_io, sep="/", names=["Name", "Value"])

df

,Name,Value
0,seller,lizengo GmbH & Co. KG
1,seller address,"Eiler Straße 3, 51107 Köln - Germany"
2,seller phone,0221 2922400
3,seller email,info@lizengo.pl
4,seller website,www.lizengo.pl
5,invoice number,140150955
6,page number,Strona 1 z 1
7,item number,Poz. 1
8,item code,Nr art. 100002-PL
9,item description,Oznaczenie Windows 7 Professional
